In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
import statsmodels.formula.api as smf

data = pd.read_csv('/workspace/all_data.csv')

In [ ]:
# Data manipulation

# Rename columns
data = data.rename(columns={
    'gradient_mean': 'grad_mean',
    'gradient_median': 'grad_med',
    'gradient_stddev': 'grad_std',
    'hm_stddev': 'hm_std',
    'hm_median': 'hm_med'
})

# Pivot and calculate edge metrics
zones_data = data.pivot_table(
    index=['WDPA_PID', 'year'], 
    columns='zone',
    values=['grad_mean', 'grad_med', 'hm_mean', 'hm_med']
).reset_index()

zones_data.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in zones_data.columns]

# Edge level calculations
zones_data['edge_level'] = zones_data['grad_mean_-1_1km'] / ((zones_data['grad_mean_-1_-3km'] + zones_data['grad_mean_1_3km']) / 2)
zones_data['edge_level_far'] = zones_data['grad_mean_-1_1km'] / ((zones_data['grad_mean_-3_-5km'] + zones_data['grad_mean_3_5km']) / 2)
zones_data['edge_level_med'] = zones_data['grad_med_-1_1km'] / ((zones_data['grad_med_-1_-3km'] + zones_data['grad_med_1_3km']) / 2)
zones_data['edge_level_med_far'] = zones_data['grad_med_-1_1km'] / ((zones_data['grad_med_-3_-5km'] + zones_data['grad_med_3_5km']) / 2)

# Calculate mean across all zones
hm_mean_cols = [col for col in zones_data.columns if col.startswith('hm_mean_')]
hm_med_cols = [col for col in zones_data.columns if col.startswith('hm_med_')]
zones_data['hm_mean_all'] = zones_data[hm_mean_cols].mean(axis=1)
zones_data['hm_med_all'] = zones_data[hm_med_cols].mean(axis=1)

# Drop zone-specific columns
cols_to_drop = [col for col in zones_data.columns if any(
    col.startswith(prefix) for prefix in ['grad_mean_', 'grad_med_', 'grad_std_', 'hm_mean_', 'hm_med_', 'hm_std_']
)]
zones_data = zones_data.drop(columns=cols_to_drop)

# Merge with other data
other_cols = [col for col in data.columns if col not in ['zone', 'grad_mean', 'grad_med', 'grad_std', 'hm_mean', 'hm_med', 'hm_std']]
df_other = data[other_cols].drop_duplicates(subset=['WDPA_PID', 'year'])

df = zones_data.merge(df_other, on=['WDPA_PID', 'year'], how='left')

# Add country data
country = pd.read_csv("https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv")
country = country.rename(columns={'Three_Letter_Country_Code': 'ISO3'})
df = df.merge(country, on='ISO3', how='left')

print(f"Loaded data: {len(df)} rows, {df['WDPA_PID'].nunique()} unique protected areas")

# Protected Area Edge Analysis

This notebook analyzes edge effects in protected areas globally from 2001-2021, examining trends in vegetation gradient intensity at PA boundaries.

## 1. Summary Statistics

In [ ]:
# Descriptivererere statistics
print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Total Protected Areas: {df['WDPA_PID'].nunique():,}")
print(f"Total Countries: {df['ISO3'].nunique()}")
print(f"Total Biomes: {df['biome'].nunique()}")
print(f"Time Period: {df['year'].min()}-{df['year'].max()}")
print(f"\nPAs by Biome:\n{df.groupby('biome')['WDPA_PID'].nunique().to_string()}")

print("\n" + "="*60)
print("PROTECTED AREA SIZE DISTRIBUTION (km²)")
print("="*60)
area_km2 = df['AREA_DISSO'] / 1e6
print(f"Min: {area_km2.min():.2f} km²")
print(f"25th percentile: {area_km2.quantile(0.25):.2f} km²")
print(f"Median: {area_km2.median():.2f} km²")
print(f"75th percentile: {area_km2.quantile(0.75):.2f} km²")
print(f"Max: {area_km2.max():.2f} km²")
print(f"Std Dev: {area_km2.std():.2f} km²")

print("\n" + "="*60)
print("EDGE LEVEL METRICS (2020)")
print("="*60)
df_2020 = df[df['year'] == 2020]
print(f"Mean Edge Level (near): {df_2020['edge_level'].mean():.4f}")
print(f"Mean Edge Level (far): {df_2020['edge_level_far'].mean():.4f}")

print("\n" + "="*60)
print("CHANGE IN EDGE LEVEL (2001-2020)")
print("="*60)
df_2001 = df[df['year'] == 2001]
change_near = df_2020['edge_level'].mean() - df_2001['edge_level'].mean()
change_far = df_2020['edge_level_far'].mean() - df_2001['edge_level_far'].mean()
print(f"Change (near): {change_near:+.4f}")
print(f"Change (far): {change_far:+.4f}")

## 2. Temporal Trends Analysis

Calculate linear regression slopes for each protected area to quantify edge level trends over time.

In [ ]:
# Calculate temporal trends for each PA
trends_list = []
for ID in df['WDPA_PID'].unique():
    df_pa = df[df['WDPA_PID'] == ID]
    X = df_pa['year'].values
    
    # Model 1: edge_level ~ year
    slope1, _, _, p1, se1 = stats.linregress(X, df_pa['edge_level'].values)
    
    # Model 2: edge_level_far ~ year
    slope2, _, _, p2, se2 = stats.linregress(X, df_pa['edge_level_far'].values)
    
    trends_list.append({
        'WDPA_PID': ID,
        'edge_level_slope': slope1,
        'edge_level_stderr': se1,
        'edge_level_tvalue': slope1 / se1,
        'edge_level_pvalue': p1,
        'edge_level_far_slope': slope2,
        'edge_level_far_stderr': se2,
        'edge_level_far_tvalue': slope2 / se2,
        'edge_level_far_pvalue': p2
    })

trends = pd.DataFrame(trends_list)
df = df.merge(trends, on='WDPA_PID', how='left')

# Classify trends
def classify_trend(slope, pvalue):
    if pvalue >= 0.05:
        return "Not Significant Increase" if slope > 0 else "Not Significant Decrease"
    return "Significantly Increase" if slope > 0 else "Significantly Decrease"

df['edge_trend'] = df.apply(lambda row: classify_trend(row['edge_level_slope'], row['edge_level_pvalue']), axis=1)
df['edge_trend_far'] = df.apply(lambda row: classify_trend(row['edge_level_far_slope'], row['edge_level_far_pvalue']), axis=1)

# Overall trend statistics
print("="*60)
print("OVERALL TRENDS (all PAs)")
print("="*60)
print(f"Mean edge_level slope: {trends['edge_level_slope'].mean():.6f}")
t_stat1, p_val1 = stats.ttest_1samp(trends['edge_level_slope'].dropna(), 0)
print(f"  t-statistic: {t_stat1:.4f}, p-value: {p_val1:.2e}")

print(f"\nMean edge_level_far slope: {trends['edge_level_far_slope'].mean():.6f}")
t_stat2, p_val2 = stats.ttest_1samp(trends['edge_level_far_slope'].dropna(), 0)
print(f"  t-statistic: {t_stat2:.4f}, p-value: {p_val2:.2e}")

# Trend distribution
edge_trend_counts = df[['WDPA_PID', 'edge_trend']].drop_duplicates()['edge_trend'].value_counts()
total_PAs = df['WDPA_PID'].nunique()

print("\n" + "="*60)
print("TREND CLASSIFICATION")
print("="*60)
for trend, count in edge_trend_counts.items():
    print(f"{trend}: {count} PAs ({count/total_PAs*100:.1f}%)")

print(f"\nSignificantly increasing: {edge_trend_counts.get('Significantly Increase', 0)/total_PAs*100:.1f}%")
print(f"Any increase (sig + non-sig): {(edge_trend_counts.get('Significantly Increase', 0) + edge_trend_counts.get('Not Significant Increase', 0))/total_PAs*100:.1f}%")

In [ ]:
## 3. Helper Functions for Statistical Analysis

In [ ]:
def run_anova_analysis(df, response_var, group_var, filter_invalid=None, title=""):
    """
    Run one-way ANOVA with Tukey HSD post-hoc test and normality check.
    
    Parameters:
    -----------
    df : DataFrame
        Input dataframe
    response_var : str
        Name of response variable (e.g., 'edge_level_slope')
    group_var : str
        Name of grouping variable (e.g., 'IUCN_CAT')
    filter_invalid : list, optional
        List of values to exclude from group_var
    title : str
        Description for output
    
    Returns:
    --------
    dict with F-statistic, p-value, and Tukey results
    """
    # Prepare data
    analysis_data = df[['WDPA_PID', group_var, response_var]].drop_duplicates()
    
    if filter_invalid:
        analysis_data = analysis_data[~analysis_data[group_var].isin(filter_invalid)]
    
    analysis_data = analysis_data.dropna(subset=[group_var, response_var])
    
    # Run ANOVA
    groups = [group[response_var].values for name, group in analysis_data.groupby(group_var)]
    f_stat, p_value = stats.f_oneway(*groups)
    
    # Tukey HSD
    tukey = pairwise_tukeyhsd(endog=analysis_data[response_var], 
                              groups=analysis_data[group_var], 
                              alpha=0.05)
    
    # Print results
    print("="*60)
    print(f"{title}")
    print("="*60)
    print(f"ANOVA: {response_var} ~ {group_var}")
    print(f"  N = {len(analysis_data)} PAs")
    print(f"  F-statistic = {f_stat:.4f}")
    print(f"  p-value = {p_value:.2e}")
    print(f"\n{tukey}")
    print()
    
    return {'f_stat': f_stat, 'p_value': p_value, 'tukey': tukey, 'data': analysis_data}


def run_linear_regression(df, response_var, predictor_var, log_transform=False, title=""):
    """
    Run OLS linear regression.
    
    Parameters:
    -----------
    df : DataFrame
        Input dataframe
    response_var : str
        Name of response variable
    predictor_var : str
        Name of predictor variable
    log_transform : bool
        Whether to log-transform predictor
    title : str
        Description for output
    
    Returns:
    --------
    Fitted model object
    """
    # Prepare data
    analysis_data = df[['WDPA_PID', predictor_var, response_var]].drop_duplicates()
    analysis_data = analysis_data.dropna(subset=[predictor_var, response_var])
    
    # Setup regression
    X = np.log(analysis_data[predictor_var]) if log_transform else analysis_data[predictor_var]
    X = sm.add_constant(X)
    y = analysis_data[response_var]
    
    # Fit model
    model = sm.OLS(y, X).fit()
    
    # Print results
    print("="*60)
    print(f"{title}")
    print("="*60)
    predictor_str = f"log({predictor_var})" if log_transform else predictor_var
    print(f"Linear Regression: {response_var} ~ {predictor_str}")
    print(f"  N = {len(analysis_data)} PAs")
    print(model.summary())
    print()
    
    return model


def plot_qq_residuals(analysis_data, response_var, group_var, title=""):
    """
    Create Q-Q plot for residuals to check normality assumption.
    
    Parameters:
    -----------
    analysis_data : DataFrame
        Data from ANOVA analysis
    response_var : str
        Name of response variable
    group_var : str
        Name of grouping variable
    title : str
        Plot title
    """
    residuals = analysis_data[response_var] - analysis_data.groupby(group_var)[response_var].transform('mean')
    
    fig, ax = plt.subplots(figsize=(6, 5))
    stats.probplot(residuals, dist="norm", plot=ax)
    ax.set_title(f'Q-Q Plot: {title}')
    plt.tight_layout()
    plt.show()

## 4. Factors Affecting Edge Trends

Analyze how edge trends vary by IUCN category, protected area size, biome, and establishment year.

### 4.1 IUCN Category

In [ ]:
# IUCN Category Analysis
invalid_iucn = ['Not Reported', 'Not Assigned', 'Not Applicable']

# Near edge trends
anova_iucn_near = run_anova_analysis(
    df, 
    response_var='edge_level_slope',
    group_var='IUCN_CAT',
    filter_invalid=invalid_iucn,
    title="IUCN CATEGORY - Near Edge Trends"
)

# Far edge trends
anova_iucn_far = run_anova_analysis(
    df, 
    response_var='edge_level_far_slope',
    group_var='IUCN_CAT',
    filter_invalid=invalid_iucn,
    title="IUCN CATEGORY - Far Edge Trends"
)

In [ ]:
# Check normality of residuals
plot_qq_residuals(anova_iucn_near['data'], 'edge_level_slope', 'IUCN_CAT', 
                  title="Near Edge Residuals by IUCN Category")

### 4.2 Protected Area Size

In [ ]:
# PA Size Analysis (log-transformed)

# Near edge trends
model_size_near = run_linear_regression(
    df,
    response_var='edge_level_slope',
    predictor_var='AREA_DISSO',
    log_transform=True,
    title="PA SIZE - Near Edge Trends"
)

# Far edge trends
model_size_far = run_linear_regression(
    df,
    response_var='edge_level_far_slope',
    predictor_var='AREA_DISSO',
    log_transform=True,
    title="PA SIZE - Far Edge Trends"
)

### 4.3 Biome

In [ ]:
# Biome Analysis

# Near edge trends
anova_biome_near = run_anova_analysis(
    df,
    response_var='edge_level_slope',
    group_var='biome',
    title="BIOME - Near Edge Trends"
)

# Far edge trends
anova_biome_far = run_anova_analysis(
    df,
    response_var='edge_level_far_slope',
    group_var='biome',
    title="BIOME - Far Edge Trends"
)

# Check normality
plot_qq_residuals(anova_biome_near['data'], 'edge_level_slope', 'biome',
                  title="Near Edge Residuals by Biome")

### 4.4 Establishment Year

In [ ]:
# Establishment Year Analysis

# Near edge trends
model_year_near = run_linear_regression(
    df,
    response_var='edge_level_slope',
    predictor_var='STATUS_YR',
    log_transform=False,
    title="ESTABLISHMENT YEAR - Near Edge Trends"
)

# Far edge trends
model_year_far = run_linear_regression(
    df,
    response_var='edge_level_far_slope',
    predictor_var='STATUS_YR',
    log_transform=False,
    title="ESTABLISHMENT YEAR - Far Edge Trends"
)

## 5. Mixed Effects Model

Test the relationship between edge level, year, and human modification, accounting for repeated measures within protected areas.

In [ ]:
# Mixed effects model: edge_level ~ year + hm_mean_all + (1|WDPA_PID)
# This accounts for repeated measurements within each PA

# Prepare data
mixed_data = df[['WDPA_PID', 'year', 'edge_level', 'edge_level_far', 'hm_mean_all']].dropna()

print("="*60)
print("MIXED EFFECTS MODEL")
print("="*60)
print(f"N observations: {len(mixed_data)}")
print(f"N protected areas: {mixed_data['WDPA_PID'].nunique()}")

# Model 1: Near edge
print("\n--- Model 1: edge_level ~ year + hm_mean_all + (1|WDPA_PID) ---")
mixed_model_near = smf.mixedlm(
    "edge_level ~ year + hm_mean_all", 
    data=mixed_data, 
    groups=mixed_data["WDPA_PID"]
).fit()
print(mixed_model_near.summary())

# Model 2: Far edge
print("\n--- Model 2: edge_level_far ~ year + hm_mean_all + (1|WDPA_PID) ---")
mixed_model_far = smf.mixedlm(
    "edge_level_far ~ year + hm_mean_all", 
    data=mixed_data, 
    groups=mixed_data["WDPA_PID"]
).fit()
print(mixed_model_far.summary())

## 6. Summary of Results

Key findings from the statistical analyses.

In [ ]:
# Generate summary statistics table
summary_results = {
    'Analysis': [],
    'Test': [],
    'Statistic': [],
    'P-value': [],
    'Interpretation': []
}

# Overall trends
summary_results['Analysis'].append('Overall Trend')
summary_results['Test'].append('One-sample t-test')
summary_results['Statistic'].append(f"t={t_stat1:.2f}")
summary_results['P-value'].append(f"{p_val1:.2e}")
summary_results['Interpretation'].append('Significant' if p_val1 < 0.05 else 'Not significant')

# IUCN
summary_results['Analysis'].append('IUCN Category')
summary_results['Test'].append('ANOVA')
summary_results['Statistic'].append(f"F={anova_iucn_near['f_stat']:.2f}")
summary_results['P-value'].append(f"{anova_iucn_near['p_value']:.2e}")
summary_results['Interpretation'].append('Trends vary by IUCN category' if anova_iucn_near['p_value'] < 0.05 else 'No difference')

# Size
summary_results['Analysis'].append('PA Size')
summary_results['Test'].append('Linear Regression')
summary_results['Statistic'].append(f"β={model_size_near.params[1]:.4f}")
summary_results['P-value'].append(f"{model_size_near.pvalues[1]:.2e}")
summary_results['Interpretation'].append('Trends vary with size' if model_size_near.pvalues[1] < 0.05 else 'No relationship')

# Biome
summary_results['Analysis'].append('Biome')
summary_results['Test'].append('ANOVA')
summary_results['Statistic'].append(f"F={anova_biome_near['f_stat']:.2f}")
summary_results['P-value'].append(f"{anova_biome_near['p_value']:.2e}")
summary_results['Interpretation'].append('Trends vary by biome' if anova_biome_near['p_value'] < 0.05 else 'No difference')

# Establishment Year
summary_results['Analysis'].append('Establishment Year')
summary_results['Test'].append('Linear Regression')
summary_results['Statistic'].append(f"β={model_year_near.params[1]:.4f}")
summary_results['P-value'].append(f"{model_year_near.pvalues[1]:.2e}")
summary_results['Interpretation'].append('Trends vary with year' if model_year_near.pvalues[1] < 0.05 else 'No relationship')

# Mixed model
summary_results['Analysis'].append('Year + Human Modification')
summary_results['Test'].append('Mixed Effects Model')
summary_results['Statistic'].append(f"β_year={mixed_model_near.params['year']:.4f}")
summary_results['P-value'].append(f"{mixed_model_near.pvalues['year']:.2e}")
summary_results['Interpretation'].append('Significant temporal effect' if mixed_model_near.pvalues['year'] < 0.05 else 'No temporal effect')

summary_df = pd.DataFrame(summary_results)

print("\n" + "="*80)
print("SUMMARY TABLE - NEAR EDGE LEVEL SLOPE ANALYSES")
print("="*80)
print(summary_df.to_string(index=False))
print("\n" + "="*80)

## 7. Export Results

In [ ]:
# Export processed data with trends
output_file = '/workspace/results/pa_edge_trends_analysis.csv'
df_export = df[['WDPA_PID', 'year', 'edge_level', 'edge_level_far', 
                'edge_level_slope', 'edge_level_pvalue', 'edge_trend',
                'edge_level_far_slope', 'edge_level_far_pvalue', 'edge_trend_far',
                'IUCN_CAT', 'AREA_DISSO', 'STATUS_YR', 'biome', 
                'ISO3', 'Country_Name', 'Continent_Name',
                'hm_mean_all', 'hm_med_all']].copy()

df_export.to_csv(output_file, index=False)
print(f"Results exported to: {output_file}")
print(f"Rows: {len(df_export)}, Columns: {len(df_export.columns)}")

# Export summary table
summary_df.to_csv('/workspace/results/statistical_summary.csv', index=False)
print(f"Summary table exported to: /workspace/results/statistical_summary.csv")